In [2]:
import pandas as pd
import numpy as np
import re
import openpyxl
import nltk
import re
import ast

# mengubah data transaksi list menjadi matriks biner (0 dan 1) agar bisa dibaca mesin
from mlxtend.preprocessing import TransactionEncoder

# # fpgrowth: algoritma pencari pola item; association_rules: Pembuat aturan "Jika-Maka"
from mlxtend.frequent_patterns import fpgrowth, association_rules

from nltk.tokenize import word_tokenize
from nltk.util import bigrams, ngrams

In [4]:
df_raw = pd.read_excel("./data/Data-Sintetis_Pengeluaran-Harian-Mahasiswa.xlsx")
df_raw.head(5)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,Top Up E-Wallet,Hiburan Digital,Keinginan,25000
1,01/03/2025,Print Tugas,Pendidikan,Kebutuhan,17000
2,01/03/2025,Nasi Padang,Makanan,Kebutuhan,28000
3,02/03/2025,Sunscreen,Skincare,Keinginan,85000
4,02/03/2025,Hangout di Mall,Nongkrong,Keinginan,132000


# Filter Daset Keinginan

In [5]:
df_desireCategory = df_raw[df_raw['Kategori'] == "Keinginan"].copy()
df_desireCategory.head(5)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,Top Up E-Wallet,Hiburan Digital,Keinginan,25000
3,02/03/2025,Sunscreen,Skincare,Keinginan,85000
4,02/03/2025,Hangout di Mall,Nongkrong,Keinginan,132000
5,02/03/2025,Moisturizer,Skincare,Keinginan,31000
7,03/03/2025,Kentang Goreng,Makanan,Keinginan,21000


# Preprocessing Data

## Data Cleaning

### Cek Missing Values

In [6]:
missing_values = df_desireCategory.isnull().sum()
print(missing_values)

Tanggal              0
Jenis Pengeluaran    0
Label                0
Kategori             0
Nominal (IDR)        0
dtype: int64


In [7]:
if missing_values.sum() > 0:
  df_missingValueClean = df_desireCategory.dropna()
  print(f"Data berhasil dibersihkan. {len(df_desireCategory) - len(df_missingValueClean)} baris dihapus.")

else:
  df_missingValueClean = df_desireCategory.copy()
  print("Tidak ada missing values")

Tidak ada missing values


In [8]:
df_missingValueClean.head(5)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,Top Up E-Wallet,Hiburan Digital,Keinginan,25000
3,02/03/2025,Sunscreen,Skincare,Keinginan,85000
4,02/03/2025,Hangout di Mall,Nongkrong,Keinginan,132000
5,02/03/2025,Moisturizer,Skincare,Keinginan,31000
7,03/03/2025,Kentang Goreng,Makanan,Keinginan,21000


### Symbol Removal

In [9]:
def removal_symbols(text):

  if pd.isna(text):
    return text

  cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  return cleaned_text

In [10]:
df_removalSymbolsClean = df_missingValueClean.copy()

In [11]:
df_removalSymbolsClean["Jenis Pengeluaran"] = df_removalSymbolsClean["Jenis Pengeluaran"].apply(removal_symbols)

In [12]:
df_removalSymbolsClean.head(5)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,Top Up EWallet,Hiburan Digital,Keinginan,25000
3,02/03/2025,Sunscreen,Skincare,Keinginan,85000
4,02/03/2025,Hangout di Mall,Nongkrong,Keinginan,132000
5,02/03/2025,Moisturizer,Skincare,Keinginan,31000
7,03/03/2025,Kentang Goreng,Makanan,Keinginan,21000


### Standarisasi Teks (lowercase)

In [13]:
def textStandardization(df, targetColumn):
  
  """
    - mengubah teks menjadi huruf kecil
    - menghapus spasi di awal dan akhir teks
  """

  df_clean = df.copy()

  for column in targetColumn:
    if column in df_clean.columns:
      df_clean[column] = df_clean[column].str.lower().str.strip()
    
    else:
      print(f"Kolom '{column}' tidak ditemukan dalam DataFrame.")
      
  return df_clean

In [14]:
df_textStandardization = df_removalSymbolsClean.copy()

In [15]:
target_cols = ["Jenis Pengeluaran", "Label", "Kategori"]

In [16]:
df_textStandardization = textStandardization(df_textStandardization, target_cols)

In [17]:
df_textStandardization.head(10)

,Tanggal,Jenis Pengeluaran,Label,Kategori,Nominal (IDR)
0,01/03/2025,top up ewallet,hiburan digital,keinginan,25000
3,02/03/2025,sunscreen,skincare,keinginan,85000
4,02/03/2025,hangout di mall,nongkrong,keinginan,132000
5,02/03/2025,moisturizer,skincare,keinginan,31000
7,03/03/2025,kentang goreng,makanan,keinginan,21000
10,03/03/2025,masker wajah,skincare,keinginan,29000
11,04/03/2025,top up ewallet,hiburan digital,keinginan,74000
12,04/03/2025,top up game pubg,hiburan digital,keinginan,95000
14,04/03/2025,jus buah,minuman,keinginan,20000
15,04/03/2025,burger,makanan,keinginan,36000


## Metadata Extraction 
-----

### Goals
- Membuat referensi/kamus yang menghubungkan Label dengan Jenis Pengeluaran

- Agar nanti saat hasil FP-Growth menunjukkan "makanan", kita bisa tahu apa aja Jenis Pengeluaran makanan

In [18]:
def extractionMetadata(
    df, 
    groupColumn, 
    detailColumn, 
    save_to_csv=True, 
    file_name="./data/extracted_metadata.csv"):
  
  """
    - mengelompokkan data berdasarkan kolom groupColumn
    - mengekstrak detail unik dari kolom detailColumn
    - menyimpan metadata ke file CSV jika save_to_csv=True
  """

  # grouping and set unik
  try:
    # mengambil data unik dalam bentuk list
    metadataDictionary = df.groupby(groupColumn)[detailColumn].apply(lambda x: list(set(x))).to_dict()
  except KeyError as e:
    print(f"error kolom: {e} tidak ditemukan dalam DataFrame.")
    return {}, pd.DataFrame()
  
  # membuat dataframe ringkasan
  df_metadataSummary = pd.DataFrame([
    {
      groupColumn: label,
      detailColumn: str(sorted(items)),
      "Jumlah Unik": len(items)
    }
    for label, items in metadataDictionary.items()
  ])
  
  # menyimpan ke CSV
  if save_to_csv:
    df_metadataSummary.to_csv(file_name, index = False, sep=';', encoding = "utf-8")

  return metadataDictionary, df_metadataSummary

In [19]:
data_dictionary, table_metadataSummary = extractionMetadata(
    df = df_textStandardization,
    groupColumn = "Label",
    detailColumn = "Jenis Pengeluaran",
    save_to_csv = True,
    file_name = "./data/extracted_metadata.csv",
)

In [20]:
table_metadataSummary.head(5)

,Label,Jenis Pengeluaran,Jumlah Unik
0,hiburan digital,"['beli skin game', 'langganan disney', 'top up...",6
1,makanan,"['batagor', 'burger', 'cilok', 'fried chicken'...",7
2,minuman,"['boba tea', 'es teh kekinian', 'jus buah', 'k...",6
3,nongkrong,"['billiard', 'bowling', 'hangout di mall', 'no...",4
4,skincare,"['lip balm', 'lipstik', 'masker wajah', 'moist...",7


## Transaction Aggregation 
---

### Tujuannya adalah mengelompokkan data berdasarkan tanggal, setiap tanggal dianggap sebagai satu transaksi

In [46]:
def aggregateTransactions(
    df, 
    groupColumn, 
    itemColumn, 
    dataFormat="%d/%m/%Y", 
    save_to_csv=True, 
    fileName="./data/aggregated_transactions.csv"):
  
  """
  Fungsi untuk mengelompokkan item berdasarkan ID transaksi (Tanggal).

  Argumens:
    df (pandas.DataFrame): DataFrame input yang berisi data transaksi.
    groupColumn (str): Nama kolom yang berisi ID transaksi (Tanggal).
    detailColumn (str): Nama kolom yang berisi item yang akan digabungkan.
    itemColumn (str): Nama kolom baru untuk menyimpan daftar item yang digabungkan.
    dataFormat (str): Format tanggal dalam kolom groupColumn.
  """

  df_aggregation = df.copy()

  # handling tanggal (parsing ke datetime)
  try:
    # mengecek apakah sudah datetime atau belum
    if not pd.api.types.is_datetime64_any_dtype(df_aggregation[groupColumn]):
      df_aggregation[groupColumn] = pd.to_datetime(df_aggregation[groupColumn], format=dataFormat, errors='coerce')
  except KeyError as e:
    print(f"gagal parsing tanggal: {e} tidak ditemukan dalam DataFrame.")

  # mengelompokkan item berdasarkan ID transaksi (Tanggal) dan item-itemnya dijadikan list
  try:
    transactionGrouped = df_aggregation.groupby(groupColumn)[itemColumn].apply(list).reset_index()
    transactionGrouped.columns = [groupColumn, "Items"]
  except KeyError as e:
    print(f"error kolom: {e} tidak ditemukan dalam DataFrame.")
    return [], pd.DataFrame()
  
  # membuat list of list (untuk input algoritma)
  transactionList = transactionGrouped["Items"].tolist()

  # menyimpan ke CSV
  if save_to_csv:
    df_save = transactionGrouped.copy()
    df_save["Items String"] = df_save["Items"].apply(lambda x: ', '.join(x))

    df_save[[groupColumn, "Items String"]].to_csv(fileName, index = False, sep=';', encoding = "utf-8")

  return transactionList, transactionGrouped


In [47]:
datasetTransaction, df_transactionAggregation = aggregateTransactions(
    df = df_textStandardization,
    groupColumn = "Tanggal",
    itemColumn = "Label",
    dataFormat = "%d/%m/%Y",
    save_to_csv = True,
    fileName = "./data/aggregated_transactions.csv"
) 

In [48]:
df_transactionAggregation.head(5)

,Tanggal,Items
0,2025-03-01,[hiburan digital]
1,2025-03-02,"[skincare, nongkrong, skincare]"
2,2025-03-03,"[makanan, skincare]"
3,2025-03-04,"[hiburan digital, hiburan digital, minuman, ma..."
4,2025-03-05,"[skincare, hiburan digital]"


## One-Hot Encoding (Transformasi Ke Matriks Biner)

# Frequent Pattern Mining (Algoritma FP-Growth)

# Creating Association Rules (Rule Generation)

# Visualisasi Report

---

- Interactive Network Graph
- The Recommendation Cards
- Sankey Diagram
- Bubble Chart Cluster
- Bar Chart "Top Pairs"